In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 56.4 MB/s eta 0:00:00
  Attempting uninstall: autograd
    Found existing installation: autograd 1.6.2
    Uninstalling autograd-1.6.2:
      Successfully uninstalled autograd-1.6.2


Importing Libraries

In [ ]:
import pennylane as qml
import numpy as np
import math

Function for position of X-Gates in Q-RAM Structure

In [ ]:
#number of Qubits provided as input
def where_to_apply_x(bin_number_length: int):

    #storing qubit positions for appending X-Gate each indices
    indices=[]

    #number of datapoints from qubits
    powers_of_two = 2 ** np.arange(bin_number_length)

    #for each datapoint positons we try to find out possible qubits to include a X-Gate
    for pos in range(2**bin_number_length):

        #store qubit positions for each indices of datapoints
        temp=[]

        #for each indices check the bit diiference and append the X-Gate
        for ind, v in enumerate(powers_of_two):
            if v & (pos ^ (pos - 1)) == v:
                temp.append(ind)
        indices.append(temp)

    return indices

Function for finding negative number

In [ ]:
def find_negative_numbers(list_number:list[int]):

    #find number of qubits w.r.t. number of elements in the list
    num_qubits = math.ceil(math.log2(len(list_number)))

    #initilize device with an extra ancilla qubit
    dev = qml.device("default.qubit", wires=num_qubits+1, shots=1024)

    #initialize the wires
    wires = list(range(num_qubits))

    #create circuit
    @qml.qnode(dev)
    def circuit(list_number):

        #apply hadamard get on each element
        for wire in wires:
            qml.Hadamard(wires=wire)

        #get the X-Gates position for corresponding qubits
        where_x = where_to_apply_x(num_qubits)

        #for every index in the number, apply the X-Gates for accessing the index of the list,
        #if the index value <0, do Flip, else nothing
        for i in range(len(list_number)):
            for j in where_x[i]:
                qml.PauliX(j)
            if list_number[i]<0:
                qml.ctrl(qml.PauliX, control=wires, control_values=[1]*len(wires))(wires=len(wires))

        #return counts of the circuit w.r.t ancilla qubit only
        return qml.counts(wires=len(wires))

    #get the counts
    counts=circuit(l)

    #if the count result contains 1, then there is -ve number
    if '1' in counts.keys():
        return True
    else:
        return False

In [ ]:
lists=[
    [1, 2, 3, 4],
    [1, 1, 1, 1],
    [0, 0, 0, 0],
    [-1, 0, 2, 3],
    [-2, 0, -2, 3],
    [-1, -4, -3, 4],
    [-1, -2, -3, -6],
    [-4.5, 2, 4, 8],
    [4,5, 6, 7, -8, 9, 2.5]
]
for l in lists:
    print(find_negative_numbers(l))


False
False
False
True
True
True
True
True
True


Reason for Validation for all numbers

In this circuit, total number of qubits is log(n)+1, where n is number of elements. For this circuit, we are not directly checking the value of the element, rather than we just check the sign, if the sign is negative, we just flip the ancilla qubit to determine, that there is a negative sign, so we can confirm, So in this case, it'll be applicable in any number.